In [3]:
import requests
from dotenv import load_dotenv
import os
import mysql.connector
import json
import csv
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import constants
import driver
import importlib
import Track_sectors
import random
import math
import pprint
from collections import OrderedDict
import tkinter as tk


In [4]:
importlib.reload(driver)
importlib.reload(constants)
importlib.reload(Track_sectors)
from driver import Driver
from constants import DB_DRIVER_NAMES, YEARS, TYRE_PARAMS, DEFAULT_BASE_LAP, DRIVER_TYRE_CONSTANTS, DRIVER_CONSTANTS
from Track_sectors import TRACK_SECTORS

In [5]:
load_dotenv()

db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')

In [6]:
quali_df = pd.read_csv('all_quali_data.csv')
race_df = pd.read_csv('all_race_data.csv')

This is the first bit of code for the GUI. Just to get it up and show users that the program is actually working its magic. 

In [7]:
root = tk.Tk()

root.geometry("1000x1000")
root.title("Formula One Race Simulator")

label = tk.Label(root, text="Formula One Race Simulator", font=('Arial', 22))
label.pack(padx=25, pady=25)



In [8]:
#This method returns the average pitstop loss for a given race. At the moment it is just the Dutch GP
#More will be added later
def get_avg_pit_loss():
    return float(21.355)

In [9]:
def get_base_time(race, year, driver):
    base_time = (quali_df[(quali_df.year == year) & (quali_df.race_name == f"{race}") & (quali_df.driver == f"{driver}")])
    if base_time.empty:
        return DEFAULT_BASE_LAP[f"{race}"]
    return float(base_time.iloc[0, 3])

In [10]:
#This method is to check weather a pit stop is now a smart option
# def time_to_pit(driver, num_laps):

#     remaining_laps = num_laps - driver.current_lap

#     # if remaining_laps < 0:
#     #     return True
#     if remaining_laps < 10 or driver.stint_lap < 6:

#         return False

#     elif driver.current_compound == "soft":

#         for i in range(1, 3):

#             if(driver.lap_times[i]+ 1 < driver.current_lap_time):
#                 print(f"first few lap times are: {driver.lap_times}")
#                 print(f"pitting because current lap time is: {driver.current_lap_time}")
#                 return True
            
#         return False
    
#     elif driver.current_compound == "medium":

#         for i in range(1, 3):

#             if(driver.lap_times[i]+ 1.9 < driver.current_lap_time):

#                 return True
            
#         return False
    
#     else:

#         for i in range(1, 3):

#             if(driver.lap_times[i]+ 2.3 < driver.current_lap_time):

#                 return True
            
#         return False

In [ ]:
def time_to_pit(driver, num_laps):
    remaining_laps = num_laps - driver.current_lap
    print(f"Driver {driver.driver_id} is on lap {driver.current_lap} and stint lap {driver.stint_lap} with laps {driver.lap_times}")
    print(f"{driver.current_lap - driver.stint_lap}")
    if remaining_laps < 10 or driver.stint_lap < 6:
        return False
    elif driver.compound == "soft":
        print(f"lap is {driver.current_lap} stint lap is {driver.stint_lap}")
        print(driver.current_lap - driver.stint_lap)
        if (driver.current_lap_time > driver.lap_times[(driver.current_lap - driver.stint_lap)] + 1):
            return True
        return False
    elif driver.compound == "medium":
        if (driver.current_lap_time > driver.lap_times[(driver.current_lap - driver.stint_lap)] + 1.9):
            return True
        return False
    else:
        if (driver.current_lap_time > driver.lap_times[(driver.current_lap - driver.stint_lap) ] + 2.3):
            return True
        return False
    
    

In [12]:
def simulate_pit_stop(current_compound, num_laps, lap_number):
    laps_remaining = num_laps - lap_number
    if current_compound == "soft":
        if laps_remaining > 25:
            next_compound = "hard"
        else:
            next_compound = "medium"
    elif current_compound == "medium":
        if laps_remaining < 13:
            next_compound = "soft"
        else:
            next_compound = "hard"
    else:
        if laps_remaining < 13:
            next_compound = "soft"
        else:
            next_compound = "medium"
    return next_compound

In [13]:
def simulate_sector_times(driver, race):
    sector_times = driver.simulate_sector_times(driver.stint_lap, race)
    return sector_times

In [14]:
def check_DRS(driver1, driver2, lap, sector_number, race):
    if lap < 3:
        return False
    elif TRACK_SECTORS[race][sector_number]["DRS"] == True:
        if driver1.race_time - driver2.race_time < 1.0:
            return True
        else:
            return False
    else:
        return False


In [15]:
def is_DRS_available(gap_to_in_front, lap):
    if lap > 3 and gap_to_in_front < 1.0:
        return True
    else:
        return False


In [16]:
def get_overtake_probability(race, sector_number, driver1, driver2):
    sector_probability = TRACK_SECTORS[race][sector_number]["overtake_probability"]
    #Interval is how much ahead the driver is going to be
    interval = driver2.race_time - driver1.race_time
    interval_factor = min(interval / 0.4, 1)
    overtake_probability = sector_probability * interval_factor
    return overtake_probability   


In [17]:
def get_following_cornering_affect2(race, sector_number, sector_time, interval):
    sector = TRACK_SECTORS[race][sector_number]
    b = 0.0015
    sector_speed_effect = 0
    sector_type = sector["type"]
    sector_speed = sector["speed"]
    sector_distance = sector["end"] - sector["start"]
    sector_distance_effect = 0
    if sector_type == "corner":
        gap_effect = (1 / (interval + 0.01))
        if sector_speed == "fast":
            sector_speed_effect = 0.25
        elif sector_speed == "medium":
            sector_speed_effect = 0.15
        else:
            sector_speed_effect = 0.1 
        sector_distance_effect = math.log(b * sector_distance + 1)
        time_loss = gap_effect * sector_speed_effect * sector_distance_effect
        if time_loss > 0.6:
            return 0.6
        else:
            return time_loss

In [18]:
time_loss = get_following_cornering_affect2("netherlands", 1, 15, 0.5)
print(time_loss)
time_loss = get_following_cornering_affect2("netherlands", 1, 15, 0.9)
print(time_loss)
time_loss = get_following_cornering_affect2("netherlands", 1, 15, 0.2)
print(time_loss)
time_loss = get_following_cornering_affect2("netherlands", 1, 15, 0)
print(time_loss)



0.1414689516802133
0.07928479709550415
0.34356745408051803
0.6


Turbelence is the main area where the project will differ from pervious ones. As we have seen in the studies up to 70% of the downforce can be lost by the car behind. This can cause massive losses in time and also increased tyre wear. Increased Tyre wear isn't something that we will look at currently (26/02/2025) but it may come in later. The factors that influence the turbulence affect are:
- How close you are to the car in front
- What type of corner is it? (Low, High, Med)
- How long is the cornering sector? 
The way to do this is to measure all of these factors and apply a custom algorithm for it. There is already a study within the notebook that looks at where drivers lose time specifically at the dutch grand prix. Notably the longer the cornering sector is the more the driver behinds loses time, in addition this is also where the faster corners are in the track and so that is a factor as well. 

In [19]:
def get_following_cornering_affect(race, sector_number, sector_time, interval):
    sector = TRACK_SECTORS[race][sector_number]
    b = 0.0015
    sector_speed = sector["speed"]
    sector_distance = sector["end"] - sector["start"]
    gap_effect = (1 / (interval + 0.0001))
    if sector_speed == "fast":
        sector_speed_effect = 0.25
    elif sector_speed == "medium":
        sector_speed_effect = 0.15
    else:
        sector_speed_effect = 0.1 
    sector_distance_effect = math.log(b * sector_distance + 1)
    time_loss = gap_effect * sector_speed_effect * sector_distance_effect
    if time_loss > 0.6:
        time_loss = 0.6
    sector_time = float(sector_time) + time_loss
    return sector_time

In [20]:
time_loss = get_following_cornering_affect("netherlands", 1, 15, 0.5)
print(time_loss)
time_loss = get_following_cornering_affect("netherlands", 1, 15, 0.9)
print(time_loss)
time_loss = get_following_cornering_affect("netherlands", 1, 15, 0.2)
print(time_loss)
time_loss = get_following_cornering_affect("netherlands", 1, 15, 0)
print(time_loss)


15.144269476818454
15.08015683297068
15.360565544012537
15.6


Slipstream works in the complete opposite as turbulence during corners. It's actually a massive benefit to have less air pressure and follow another car during straight sections. Since instead of using the air for downforce the cars actually want less air to generate less drag. This is also how the Drag Reduction System (DRS) works. It isn't half as strong as the turbulence's affect on cornering. Sometimes with different car setups and energy system uses the slipsteam affect is cancelled out. Making some drivers only gain during the cornering sectors. 

In [21]:
def get_slipstream_affect(race, sector_number, sector_time, interval):
    sector = TRACK_SECTORS[race][sector_number]
    time_loss = 0 
    sector_distance = sector["end"] - sector["start"]
    if 0 < interval < 1.2:
        time_gain = (-0.24 * interval + 0.3) * (sector_distance / 750)
        return sector_time - time_gain
    else:
        return sector_time

In [22]:
slipstream_gain = get_slipstream_affect("netherlands", 2, 9.4, 0.8)
print(slipstream_gain)
slipstream_gain = get_slipstream_affect("netherlands", 2, 9.4, 1.2)
print(slipstream_gain)
slipstream_gain = get_slipstream_affect("netherlands", 2, 9.4, 1.19)
print(slipstream_gain)
slipstream_gain = get_slipstream_affect("netherlands", 2, 9.4, 22)
print(slipstream_gain)
slipstream_gain = get_slipstream_affect("netherlands", 2, 9.4, 0.01)
print(slipstream_gain)

9.3064
9.4
9.38752
9.4
9.14208


In [23]:
#Speed is measured in m/s
def get_DRS_affect(race, sector_number, sector_time):
    sector = TRACK_SECTORS[race][sector_number]
    distance = sector['end'] - sector['start']
    original_speed = distance / sector_time
    drs_speed = original_speed + 3.6
    new_sector_time = distance / drs_speed
    return new_sector_time

In [24]:
def get_tyre_traction_level(compound):
    if compound == "soft":
        return 1.0
    elif compound == "medium":
        return 0.9
    else:
        return 0.8

In [25]:
def calculate_starting_sector(driver):
    #This is the average acceleratin of an f1 car
    base_acceleration = 10.31630769 
    driver_skill = DRIVER_CONSTANTS[driver.driver_id]['starting']
    #If the random number is higher than the drivers skill then give them a bad start
    if (random.uniform(0, 1.05) > driver_skill):
        random_multiplier = random.uniform(0.95, 0.99)
        #print(f"Driver {driver.driver_id} has had a bad start")
        base_acceleration *= random_multiplier
    tyre_traction = get_tyre_traction_level(driver.compound)
    varied_acceleration = base_acceleration * tyre_traction
    sector_time = (((driver.starting_distance * 2)/ varied_acceleration) ** 0.5)
    return sector_time

In [26]:
def assign_strategy(driver, best_strategy_dictionary):
    best_tyre = list(best_strategy_dictionary[driver].keys())[0]
    return best_tyre

In [ ]:
def simulate_race(race, year, num_laps, best_strategy_dictionary):
    drivers = []
    compounds = ["soft", "medium", "hard"]
    number_of_sectors = 0
    for driver in DB_DRIVER_NAMES:
        if best_strategy_dictionary == None:
            starting_compound = compounds[random.randint(0,2)]
        else:
            #If we have figured out a best average strategy then this is where to assign it.
            starting_compound = assign_strategy(driver, best_strategy_dictionary)
        drivers.append(Driver(driver_id=driver,
                              base_lap_time=get_base_time(race, year, driver),
                              fuel_level = 110,
                              compound=starting_compound, race_name=race))
    drivers.sort(key= lambda d: d.base_lap_time)
    for idx, driver in enumerate(drivers):
        driver.position = idx + 1
        driver.grid_slot = driver.position
        driver.starting_distance = driver.get_starting_distance(race)
        driver.starting_sector = calculate_starting_sector(driver)
        driver.compounds.append(driver.starting_compound)
        #print(f"Driver {driver.driver_id} has starting sector {driver.starting_distance} completed in time {driver.starting_sector}")

        
    for lap in range(0, num_laps):
        isolated_sectors = {}
        print(f"Simulating Lap {lap}")
        for driver in drivers:
            driver.current_lap_time = 0
            driver.stint_lap += 1
            sector_times = simulate_sector_times(driver, race)
            isolated_sectors[driver.driver_id] = {
                "lap": lap,
                "sectors": sector_times
            }
            number_of_sectors = len(isolated_sectors[driver.driver_id]['sectors'])
        for i in range(number_of_sectors):
            #print(f"Simulating Sector {i}")
            for j in range(0, len(drivers)):
                is_DRS = check_DRS(drivers[j], drivers[j-1], lap, i, race)
                drivers[j].drs = is_DRS
                drivers[j].interval = interval = drivers[j].race_time - drivers[j-1].race_time
                #print(f"{drivers[j].driver_id} is in position {drivers[j].position}. Gap to car in front:{interval:.3f} with race time {drivers[j].race_time} on compound {drivers[j].compound}")
            #Need to refactor this section to become more efficient in detemining what effects to apply
            for driver in drivers:
                if (lap == 0 and i == 0):
                    driver.race_time = driver.starting_sector
                    driver.current_lap_time = driver.starting_sector
                else:      
                    sector_time = isolated_sectors[driver.driver_id]['sectors'][i]
                    if TRACK_SECTORS[race][i]["type"] == "corner":
                        sector_time = get_following_cornering_affect(race, i, sector_time, driver.interval)
                    else:
                        if driver.drs:
                            sector_time = get_DRS_affect(race, i, sector_time)
                            driver.drs = False
                        else:
                            sector_time = get_slipstream_affect(race, i, sector_time, driver.interval)
                    driver.race_time += sector_time
                    driver.current_lap_time += sector_time
            #Overtake checking
            for j in range(1, len(drivers)):
                if (drivers[j].race_time < drivers[j-1].race_time):
                    # print(f"overtake will occur on lap {lap} sector {i} between driver {drivers[j].driver_id} and {drivers[j-1].driver_id}")
                    probability = get_overtake_probability(race, i, drivers[j], drivers[j-1])
                    #print(f"Probability of overtake {probability}")
                    if random.random() > probability:
                        drivers[j].race_time += 0.5
                        #print(f"{drivers[j].driver_id} overtake on {drivers[j-1].driver_id} has failed")
                    else:
                        #print(f"{drivers[j].driver_id} has overtaken {drivers[j-1].driver_id}")
                        drivers[j].overtakes += 1
                        continue
            #Sort the drivers by race time
            drivers.sort(key=lambda d: d.race_time)
            for idx, driver in enumerate(drivers):
                new_position = idx + 1
                #if driver.position != new_position:
                    #print(f"Driver {driver.driver_id} moved to position {new_position}")
                driver.position = new_position
            #print("\n")
        for driver in drivers:
            driver.lap_times.append(driver.current_lap_time)
            is_pitting = time_to_pit(driver, num_laps)
            if is_pitting:
                new_compound = simulate_pit_stop(driver.compound, num_laps, lap)
                driver.compound = new_compound
                driver.race_time += get_avg_pit_loss() #Needs changing in the future
                #print(f"Driver {driver.driver_id} is pitting on lap {lap}")
                driver.pit_laps.append(lap)
                driver.compounds.append(new_compound)
                driver.stint_lap = 1
            drivers.sort(key=lambda d: d.race_time)
            for idx, driver in enumerate(drivers):
                new_position = idx + 1
                #if driver.position != new_position:
                    #print(f"Driver {driver.driver_id} moved to position {new_position}")
                driver.position = new_position
    return drivers


In [28]:
def order_drivers(drivers):
    drivers.sort(key=lambda d: d.race_time)
    for idx, driver in enumerate(drivers):
        new_position = idx + 1
        driver.position = new_position

    return drivers

In [29]:
#The input here must be ordered correctly by race time
def calculate_gaps_to_in_front(drivers):
    for i in range(0, len(drivers)):
        interval = drivers[i].race_time - drivers[i-1].race_time
        drivers[i].gap_to_in_front = interval

In [48]:
def calculate_overtakes(drivers, sector_number, race):
    for i in range(1, len(drivers)):
        if (drivers[i].race_time < drivers[i-1].race_time):
            print(f"Checking for overtake between {drivers[i].driver_id} and {drivers[i-1].driver_id} during sector {sector_number} with gap {drivers[i].race_time - drivers[i-1].race_time}")
            overtaking_probability = get_overtake_probability(race, sector_number, drivers[i], drivers[i-1])
            if random.random() > overtaking_probability:
                #The overtake has failed
                drivers[i].race_time += 0.5
                print(f"Driver {drivers[i].driver_id} has failed an overtake so 0.5 seconds has been added \n")
                #print(f"Driver {drivers[i].driver_id} could not overtake {drivers[i-1].driver_id}")
            else:
                #The overtak succeeds
                drivers[i].overtakes += 1
                #print(f"Driver {drivers[i].driver_id} has overtaken {drivers[i-1].driver_id}")
    drivers = order_drivers(drivers)
    return drivers

In [42]:
def simulate_the_race(race, year, num_laps, best_strategy_dictionary):
    #Assign the drivers and their starting variables
    
    drivers = []
    compounds = ["soft", "medium", "hard"]
    number_of_sectors = 0
    for driver_name in DB_DRIVER_NAMES:
        if best_strategy_dictionary == None:
            starting_compound = compounds[random.randint(0,2)]
        else:
            starting_compound = assign_strategy(driver_name, best_strategy_dictionary)
        drivers.append(Driver(driver_id=driver_name, base_lap_time=get_base_time(race, year, driver_name),
                              fuel_level=110, compound= starting_compound,
                              race_name = race))
    
    drivers.sort(key= lambda d: d.base_lap_time)
    for idx, driver in enumerate(drivers):
        driver.position = idx+1
        driver.grid_slot = driver.position
        driver.starting_distance = driver.get_starting_distance(race)
        driver.starting_sector = calculate_starting_sector(driver)
        driver.compounds.append(driver.starting_compound)


    #simulate an isolated lap

    #remember we index from 0
    for lap in range(0, num_laps):
        print(f"Simulating lap {lap}")
        isolated_sectors = {}
        for driver in drivers:
            driver.current_lap_time = 0
            sector_times = simulate_sector_times(driver, race)
            isolated_sectors[driver.driver_id] = {
                "lap": lap,
                "sectors": sector_times
            }
        number_of_sectors = len(isolated_sectors[driver.driver_id]['sectors'])
        
        #Loop through each sector
        for sector in range(0, number_of_sectors):
            print(f"Simulating sector {sector} of lap {lap}")
            for driver in drivers:    
                #If this is the first sector of the first lap calculate the starting sector 
                if (lap == 0 and sector == 0):
                    driver.current_lap_time += driver.starting_sector
                    driver.race_time += driver.starting_sector
                    print(f"Driver {driver.driver_id} has a starting sector of {driver.starting_sector}")
                #If this is not the first lap of the first sector we continue as normal
                else:
                    #For each sector it is crucial we calculate the gap to the driver in front
                    calculate_gaps_to_in_front(drivers)
                    current_sector_type = TRACK_SECTORS[race][sector]["type"]
                    drs_sector = TRACK_SECTORS[race][sector]["DRS"]
                    #print(isolated_sectors)
                    isolated_sector_time = isolated_sectors[driver.driver_id]['sectors'][sector]
                    #print(f"Driver {driver.driver_id} has isolated sector time {isolated_sector_time} for sector {sector}")
                    #print(isolated_sector_time)
                    #Is it a straight? Apply affects
                    if current_sector_type == "straight":
                        #is it a DRS sector?
                        if drs_sector:
                            #Check if the driver should get drs
                            driver.drs = is_DRS_available(driver.gap_to_in_front, lap)
                            if driver.drs:
                                recalculated_sector_time = get_DRS_affect(race, sector, isolated_sector_time)
                                driver.drs = False
                        #We have calculated the drs affect where needed, but no matter what there will alwways be a slipstream affect so we apply both in some cases. 
                        recalculated_sector_time = get_slipstream_affect(race, sector, recalculated_sector_time, driver.gap_to_in_front)
                    #Is it a corner? Apply affects
                    else:
                        recalculated_sector_time = get_following_cornering_affect(race, sector, isolated_sector_time, driver.gap_to_in_front)
                    print(f"driver {driver.driver_id} has sector time {recalculated_sector_time} for sector {sector} with race time {driver.race_time}")
                    driver.current_lap_time += recalculated_sector_time
                    driver.race_time += recalculated_sector_time
                #Process the overtaking probability
            drivers = calculate_overtakes(drivers, sector, race)
            #Lap simulation complete HERE
        #Do we need to pit
        is_it_time_to_pit = time_to_pit(driver, num_laps)
        if is_it_time_to_pit:
            #simulate pit stop
            new_compound = simulate_pit_stop(driver.compound, num_laps, lap)
            driver.compound = new_compound
            driver.compounds.append(new_compound)
            driver.pit_laps.append(lap)
            driver.race_time += get_avg_pit_loss()
            driver.stint_lap = 0
            driver.current_lap_time += get_avg_pit_loss()
        #order drivers at the end of the lap
        drivers = order_drivers(drivers)
        for driver in drivers:
            print(f"Driver {driver.driver_id} completed lap {driver.current_lap} with time {driver.current_lap_time} \n")
            driver.lap_times.append(driver.current_lap_time)
            driver.stint_lap += 1
            driver.current_lap += 1
    return drivers


So the code works obviously. It will be and has been improved over time. The way to use this simulation is to simulate a huge number of races and determine what is the best strategy. The idea is that we gather an average time for each strategy and then see what happens. With a large enough number of simulations we can probably choose a random tyre compound to pit onto. Since eventually we get the ideal strategy. 

In [32]:
compounds = ["soft", "medium", "hard"]

In [33]:
results = {}
for driver in DB_DRIVER_NAMES:
    results[driver] = {}
    for compound in compounds:
        results[driver][compound] = []
for i in range(100):
    simulated_drivers = simulate_race("netherlands", 2024, 71, None)
    for driver in simulated_drivers:
        results[driver.driver_id][driver.compound].append(driver.race_time)


Simulating Lap 0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Simulating Lap 1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Simulating Lap 2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Simulating Lap 3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Simulating Lap 4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Simulating Lap 5
0
lap is 6 stint lap is 6
0
0
lap is 6 stint lap is 6
0
0
lap is 6 stint lap is 6
0
0
lap is 6 stint lap is 6
0
0
lap is 6 stint lap is 6
0
0
lap is 6 stint lap is 6
0
0
lap is 6 stint lap is 6
0
0
lap is 6 stint lap is 6
0
0
0
0
lap is 6 stint lap is 6
0
0
0
lap is 6 stint lap is 6
0
0
0
0
0
lap is 6 stint lap is 6
0
0
0
0
0
0
Simulating Lap 6
0
lap is 7 stint lap is 7
0
0
lap is 7 stint lap is 7
0
0
lap is 7 stint lap is 7
0
0
lap is 7 stint lap is 7
0
0
lap is 7 stint lap is 7
0
0
lap is 7 stint lap is 7
0
0
lap is 7 stint lap is 7
0
0
lap is 7 stint lap is 7
0
0
0
0
lap is 7 stint lap is 7
0
0
0
lap is 7 stint lap is 7
0
0
0
0
0
lap is 7 stint lap is 7
0
0
0
0
0
0
S

25
25
26
23
27
27
26
0
0
28
0
0
0
0
28
0
0
0
30
27
0
0
Simulating Lap 40
25
25
26
23
27
27
26
0
28
0
0
0
0
0
28
0
0
0
30
27
0
0
Simulating Lap 41
25
25
26
23
27
27
26
28
0
0
0
0
0
0
28
0
0
30
0
27
0
0
Simulating Lap 42
25
25
26
23
27
27
26
28
0
0
0
0
0
0
28
0
0
30
27
0
0
0
Simulating Lap 43
25
25
26
23
27
27
26
28
0
0
0
0
0
0
28
0
0
30
27
0
0
0
Simulating Lap 44
25
25
26
23
27
27
26
28
0
0
0
0
0
28
0
0
0
30
27
0
0
0
Simulating Lap 45
25
25
26
23
27
27
26
28
0
0
0
0
0
28
0
0
0
30
27
0
0
0
Simulating Lap 46
25
25
26
23
27
27
26
28
0
0
0
0
0
0
46
0
0
30
27
0
0
0
Simulating Lap 47
25
25
26
23
27
27
26
28
0
0
0
28
0
0
46
0
30
0
27
0
0
0
Simulating Lap 48
25
25
26
23
27
27
26
28
0
28
0
0
46
0
48
30
0
0
27
0
0
0
Simulating Lap 49
25
25
26
23
27
27
26
28
0
28
0
0
46
48
0
30
0
27
0
0
0
49
Simulating Lap 50
25
25
26
23
27
27
26
28
0
28
0
46
0
50
49
30
0
27
0
0
49
0
Simulating Lap 51
25
25
26
23
27
27
26
28
0
28
46
0
48
50
49
30
27
0
0
49
0
50
Simulating Lap 52
25
25
26
23
27
27
26
28
0
28
46
48


In [34]:
#This is where we calculate the average times
average_times = {}
best_average_times = {}
sorted_best_average_times = {}

for driver in DB_DRIVER_NAMES:
    average_times[driver] = {}
    for compound in compounds:
        average_time = sum(results[driver][compound]) / len(results[driver][compound])
        average_times[driver][compound] = average_time
for driver in average_times:
    best_average_times[driver] = {}
    best_strategy = min(average_times[driver], key=average_times[driver].get)
    best_average_times[driver][best_strategy] = average_times[driver][best_strategy]
    sorted_best_average_times = OrderedDict(sorted(best_average_times.items(), key=lambda x: list(x[1].values())[0]))
pprint.pprint(sorted_best_average_times)

ZeroDivisionError: division by zero

In [35]:
def best_strategy_calculator():       
    ideal_drivers = simulate_race("netherlands", 2024, 71, None)
    for i in range(60):    
        secondary_drivers = simulate_race("netherlands", 2024, 71, None)
        for j in range(len(ideal_drivers)):
            if ideal_drivers[j].race_time > secondary_drivers[j].race_time:
                ideal_drivers[j] == secondary_drivers[j]
    for driver in ideal_drivers:
        print(f"Driver {driver.driver_id} has the best strategy starting on the {driver.starting_compound} with race time {driver.race_time}")


Now that we have the best average strategy. We are going to use these and simulate a large size of races with each driver using their best average strategy. 

In [ ]:
# def write_final_data_to_csv(drivers, simulation_number):
#     if drivers:
#         file_path = 'results.csv'
#         file_exists = os.path.exists(file_path) and os.path.getsize(file_path) > 0
#         with open('results.csv', 'a', newline='') as csvfile:
#             fieldnames = ["race_number", "Driver", "starting_position", "finishing_position", "race_time", "strategy", "pit_laps", "overtakes", "lap_times", "number_of_laps_completed"]
#             writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#             if not file_exists:
#                 writer.writeheader()
#             for driver in drivers:
#                 writer.writerow({"race_number": simulation_number, "Driver": driver.driver_id, "starting_position": driver.grid_slot, "finishing_position": driver.position,
#                                 "race_time": driver.race_time, "strategy": driver.compounds, "pit_laps": driver.pit_laps, "overtakes": driver.overtakes, "lap_times": driver.lap_times, "number_of_laps_completed": len(driver.lap_times)})

In [ ]:
# testing_drivers = simulate_the_race("netherlands", 2024, 71, None)
# write_final_data_to_csv(testing_drivers, 1)

Simulating lap 0
Simulating sector 0 of lap 0
Driver lando-norris has a starting sector of 7.382302895846258
Driver max-verstappen has a starting sector of 7.952947483291129
Driver oscar-piastri has a starting sector of 8.073911802354596
Driver george-russell has a starting sector of 7.328122810783946
Driver sergio-perez has a starting sector of 8.310560031227086
Driver charles-leclerc has a starting sector of 7.726730426460732
Driver fernando-alonso has a starting sector of 7.638992827490151
Driver lance-stroll has a starting sector of 8.864587992352272
Driver carlos-sainz-jr has a starting sector of 8.263446712331538
Driver lewis-hamilton has a starting sector of 9.016755435559656
Driver yuki-tsunoda has a starting sector of 8.983177430837825
Driver pierre-gasly has a starting sector of 8.130739443574772
Driver nico-hulkenberg has a starting sector of 9.196457632933583
Driver kevin-magnussen has a starting sector of 9.301263952525835
Driver daniel-ricciardo has a starting sector of 8

IndexError: list index out of range

In [ ]:
refined_results = {}
for driver in DB_DRIVER_NAMES:
    refined_results[driver] = {}
    for compound in compounds:
        refined_results[driver][compound] = []
for i in range(200):
    simulated_drivers = simulate_race("netherlands", 2024, 71, sorted_best_average_times)
    for driver in simulated_drivers:
        refined_results[driver.driver_id][driver.compound].append(driver.race_time)
    
    

In [ ]:

average_refined_results = {}
for driver in DB_DRIVER_NAMES:
    average_time_for_strategy = sum(results[driver][compound]) / len(results[driver][compound])
    average_refined_results[driver] = average_time_for_strategy


sorted_dict= sorted(average_refined_results.items(), key=lambda x:x[1])
pprint.pprint(sorted_dict)

[('lando-norris', 5527.909858219066),
 ('max-verstappen', 5588.60280626367),
 ('oscar-piastri', 5618.770756412906),
 ('george-russell', 5633.148859332466),
 ('lance-stroll', 5699.329057780033),
 ('charles-leclerc', 5713.585629067769),
 ('yuki-tsunoda', 5717.684709495915),
 ('pierre-gasly', 5721.53367331155),
 ('nico-hulkenberg', 5727.715835448189),
 ('lewis-hamilton', 5728.32198384872),
 ('sergio-perez', 5742.621111999958),
 ('esteban-ocon', 5776.951165678088),
 ('daniel-ricciardo', 5781.018543449982),
 ('kevin-magnussen', 5792.512046719278),
 ('valtteri-bottas', 5832.637536852935),
 ('logan-sargeant', 5856.536507498256),
 ('franco-colapinto', 5864.330553537689),
 ('liam-lawson', 5875.387904167824),
 ('carlos-sainz-jr', 5875.4693664868855),
 ('guanyu-zhou', 5887.646025553897),
 ('alexander-albon', 5940.271443449944),
 ('fernando-alonso', 9711.605239012248)]


Now we need to design a format that shows our results in a fashion that is helpful.
